# Happywhale - CS795 Project 1
***
Old Dominion University

2/26/2022
#### Authors: Raphael J. Sandor, Xiangrui Xu


# Imports

In [ ]:
import os
import keras
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from keras.optimizers import adam_v2
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import train_test_split
import PIL
from PIL import Image
import warnings
import pathlib
import tensorflow as tf
from pathlib import Path
warnings.filterwarnings('ignore')
%matplotlib inline

# Setting Up the Data

In [ ]:
train_df = pd.read_csv('train.csv')
# Use sample submission to gather which images need predictions made
test_df = pd.read_csv('sample_submission.csv')
total_image_df =  pd.concat([train_df['image'], test_df['image']])
TRAIN_IMAGES_DIR = Path("./train_images")
TEST_IMAGES_DIR = Path("./test_images")
train_images = list(TRAIN_IMAGES_DIR.glob('./*'))

# Exploring the dataset

In [ ]:
img = PIL.Image.open(str(train_images[1000]))
plt.figure(figsize = (10,10))
imgplot = plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
trainImgCnt = len(list(TRAIN_IMAGES_DIR.glob('*.jpg')))
testImgCnt = len(list(TEST_IMAGES_DIR.glob('*.jpg')))
print("Train image count: " + str(trainImgCnt))
print("Test image count: " + str(testImgCnt))

In [ ]:
train_df.info()

In [ ]:
train_df.head()

## List of unique species 

In [ ]:
species = train_df['species']

In [ ]:
train_df.describe()

In [ ]:
unique_species = pd.unique(species) # returns numpy.ndarray 
print(unique_species)

# Whales and Dolphins

In [ ]:
# Note that the dataset includes some typo mistakes mistakes.
train_df['species'].replace('bottlenose_dolpin', 'bottlenose_dolphin', inplace=True)
train_df['species'].replace('kiler_whale', 'killer_whale', inplace=True)
train_df['species'][(train_df['species'] =="pilot_whale") | (train_df['species'] =="globis" )]='short_finned_pilot_whale'
print("Total species after :",len(train_df.species.unique()))

whales = ['humpback_whale','beluga','minke_whale', 'fin_whale', 'blue_whale', 'gray_whale',
          'southern_right_whale','sei_whale', 'cuviers_beaked_whale', 'brydes_whale']

dolphins = ['melon_headed_whale','false_killer_whale', 'bottlenose_dolphin', 'common_dolphin', 
            'dusky_dolphin', 'killer_whale', 'long_finned_pilot_whale', 'spinner_dolphin', 
            'spotted_dolphin','commersons_dolphin', 'white_sided_dolphin', 'short_finned_pilot_whale',
            'rough_toothed_dolphin', 'pantropic_spotted_dolphin', 'frasiers_dolphin', 'pygmy_killer_whale']

In [ ]:
whale_df = train_df[train_df['species'].str.contains('|'.join(whales))]
dolphin_df = train_df[train_df['species'].str.contains('|'.join(dolphins))]

In [ ]:
## lets check speiceis in visually
# Nikhil Jothi Prakash 
# Works cited: https://www.kaggle.com/nikhiljothiprakash/happy-whale-and-dolphin
plt.figure(figsize=(16, 12))
plt.rcParams["font.size"] = 16
plt.barh(train_df["species"].value_counts().sort_values(ascending=True).index,train_df["species"].value_counts().sort_values(ascending=True),tick_label = train_df["species"].value_counts().sort_values(ascending=True).index)
plt.show()

In [ ]:
print(whale_df)
print(dolphin_df)

In [ ]:
print("Species of whales: " + str(len(whales)))
print("Species of dolphins: " + str(len(dolphins)))

In [ ]:
print("Number of whales: " + str(whale_df.shape[0]))
print("Number of dolphins: " + str(dolphin_df.shape[0]))

In [ ]:
# now lets classify on whales.
print(train_df.species.isin(whale_df.species).astype(int))
train_df['isWhale'] = train_df.species.isin(whale_df.species).astype(int)
train_data = []

In [ ]:
train_df.describe()

In [ ]:
# Borrowed from 
# https://www.kaggle.com/samir95/species-classification
_, dev, _ , _ = train_test_split(train_df, train_df['isWhale'], test_size=0.1)

dev.shape

# Resize images
See if image resizing is effective

In [ ]:
import fastai
from fastai.vision.all import *
from fastai.basics import *
from fastai.data.all import *

from fastai.vision.core import *
import fastbook
fastbook.setup_book() 
pd.options.mode.chained_assignment = None  # default='warn'
resizedPath = Path("train_images256_256")
dev['imagePath'] = dev['image'].apply(lambda f: TRAIN_IMAGES_DIR/f)

dblock = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                   get_x=ColReader('imagePath'),
                   get_y=lambda r: r['isWhale'],
                   splitter=RandomSplitter(seed=42),
                   item_tfms=Resize(460),
                   batch_tfms=aug_transforms(size=224))
#dblock.summary(dev)
dsets = dblock.datasets(dev)
dls = dblock.dataloaders(dev)

# Start CNN Learning

In [ ]:
dls.show_batch(max_n=20)
learn = cnn_learner(dls, resnet34, metrics=[accuracy, error_rate])
learn.lr_find()

# Fine Tune and Improve

In [ ]:
learn.fine_tune(2, base_lr=3e-3)

In [ ]:
learn.freeze()
learn.fit_one_cycle(3, 3e-3)

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.fit_one_cycle(6, lr_max=slice(1e-6, 1e-4))

# Classification based on Individual 
#### using research by Andre C Ferreira as a starting point on this venture
##### https://github.com/AndreCFerreira/Bird_individualID/blob/master/Train_CNN/TRAIN_CNN.ipynb

Lets explore how many individuals we are working with in this

In [ ]:
# Works cited:
# https://www.kaggle.com/nikhiljothiprakash/happy-whale-and-dolphin/notebook
concat_df = pd.concat([train_df['image'], test_df['image']])
concat_df.head()


In [ ]:
## MAY NOT NEED THIS WILL REMOVE OTHERWISE

# Works cited https://www.kaggle.com/noirchannel/myeda
labels = train_df["individual_id"].unique()
labels = {ind: idx for idx, ind in enumerate(labels)}